# Métrica 4_2_02 — Beneficios privados provenientes de sectores con posible conflicto de interés
| **Escenario**                                                          | **Condición exacta evaluada**                                                                 | **Resultado**    |
| ---------------------------------------------------------------------- | --------------------------------------------------------------------------------------------- | ---------------- |
| **No hay beneficios privados declarados**                              | La lista `beneficiosPrivados.beneficio` no existe o está vacía                                | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                            | No existe: <br>• nombre del ente público, o <br>• sector del beneficio                        | ⚪ **SIN_DATO**   |
| **El beneficio proviene de un sector marcado como T, D o C**           | `sector ∈ sectores_riesgo[funcion_principal_del_ente]`                                        | 🔴 **NO_CUMPLE** |
| **El beneficio NO proviene de sectores T, D o C**                      | `sector` **no** se encuentra en los sectores riesgo asociados a la función principal del ente | 🟢 **CUMPLE**    |
| **La lista de beneficios existe pero los campos están vacíos o nulos** | Sectores vacíos, sin identificar o sin clasificar                                             | ⚪ **SIN_DATO**   |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata


METRIC_ID = "4_2_02_SECTOR_RIESGO_BENEFICIOS"


# -------------------------------------------------------------------
# MATRIZ TEMPORAL (reemplazar por tu catálogo oficial)
# -------------------------------------------------------------------
# Esta matriz indica: FUNCION PRINCIPAL DEL ENTE PÚBLICO -> SECTORES RIESGO
# Ejemplo: {"SALUD": ["T", "D"], ...}
MATRIZ_SECTORES_FUNCIONES = {
    # 🔴 Rellenar con el catálogo real
    "SALUD": ["T", "D"],
    "OBRAS PUBLICAS": ["T", "C"],
    "EDUCACION": ["D"],
    "SEGURIDAD": ["T", "C"],
}


# -------------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------------
def limpiar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    # remover acentos
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    # quitar dobles espacios
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------------
# EVALUADOR DE LA MÉTRICA
# -------------------------------------------------------------------
def evaluar_metrica(doc):

    # 🅐 Nombre del ente público
    ente = limpiar(
        extraer(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    if not ente:
        return "SIN_DATO"

    # Determinar función principal según tu catálogo (esto debe venir de catálogo real)
    funcion_principal = ente  # 🔴 Temporal: tú lo ajustas según tu catálogo real

    sectores_riesgo = MATRIZ_SECTORES_FUNCIONES.get(funcion_principal, [])

    # 🅑 Beneficios privados
    beneficios = extraer(
        doc, "declaracion.intereses.beneficiosPrivados.beneficio"
    )

    if not beneficios:
        return "NO_APLICA"

    if not isinstance(beneficios, list):
        beneficios = [beneficios]

    hubo_datos = False

    for ben in beneficios:
        sector = limpiar(ben.get("sector"))
        beneficiario = limpiar(ben.get("beneficiario"))

        if not sector:
            return "SIN_DATO"

        hubo_datos = True

        # Validar si el sector está marcado como T, D o C
        if sector in sectores_riesgo:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "SIN_DATO"

    return "CUMPLE"


# -------------------------------------------------------------------
# PROCESADOR MONGO
# -------------------------------------------------------------------
def procesar_metrica_4_2_02():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1,
            "declaracion.intereses.beneficiosPrivados": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"⚠️ Error en documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_02()



🚀 Procesando métrica 4_2_02_SECTOR_RIESGO_BENEFICIOS...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > 2000 documentos procesados...
 > 4000 documentos procesados...
 > 6000 documentos procesados...
 > 8000 documentos procesados...
 > 10000 documentos procesados...
Conexión cerrada.


Traceback (most recent call last):
  File "C:\Users\Mauro\AppData\Local\Temp\ipykernel_56520\2556856645.py", line 149, in procesar_metrica_4_2_02
    tgt.bulk_write(operaciones)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\_csot.py", line 125, in csot_wrapper
    return func(self, *args, **kwargs)
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py", line 791, in bulk_write
    bulk_api_result = blk.execute(write_concern, session, _Op.INSERT)
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\bulk.py", line 751, in execute
    return self.execute_command(generator, write_concern, session, operation)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\bulk.py", line 604, in execute_command
    _ = client._retryable_write(
        self.is_retryable,
    ...<4 lines>...
        operation_id=op_id